# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages
import re
import os
import csv
import glob
import json
import cassandra
import numpy as np
import pandas as pd

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(f"Current Dir - {os.getcwd()}")

# Get your current folder and subfolder event data
filepath = os.path.join(os.getcwd(), 'event_data')

# Create a glob listing all the CSVs from filepath
file_path_list = glob.glob(os.path.join(filepath,'*.csv'))

Current Dir - /home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
try:
    # To establish connection and begin executing queries, need a session
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    raise Exception(e)

In [9]:
#Keyspace details
keyspace_name = 'sparkify'
replication_strat = {'class' : 'SimpleStrategy', 'replication_factor' : 1 }
create_keyspc = """CREATE KEYSPACE IF NOT EXISTS {0} WITH REPLICATION = {1}""".format(keyspace_name, replication_strat)

#### Create Keyspace

In [6]:
# Creating a Keyspace 
try:
    session.execute(create_keyspc)
except Exception as e:
    raise Exception(e)

#### Set Keyspace

In [11]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace(keyspace_name)
except Exception as e:
    raise Exception(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1:
> The artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


Query for the data would look like:
```
select 
    artist, 
    song_title, 
    song_length 
from 
    session_songs 
where 
    sessionId = 338 
    and itemInSession = 4
```

Plan:
1. As can be seen we require sessionId and itemInSession in our where clause so we know what both those are gonna make our composite primary key.
2. sessionId would be the partition key and itemInSession would be clustering key. Mainly because sessionId would be unique for every session.
3. Now that we know our primary keys we will also need artist name, song name and song length to satisfy the query.

In [12]:
create_query1 = """CREATE TABLE IF NOT EXISTS session_songs
(sessionId int, itemInSession int, artist text, song_title text, song_length float,
PRIMARY KEY(sessionId, itemInSession))"""

try:
    session.execute(create_query1)
except Exception as e:
    raise Exception(e)

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        # wrapping casts to bypass python data defs and feed data to cassandra
        try:
            session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))
        except Exception as e:
            raise Exception(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
val_query1 = """select artist, song_title, song_length from session_songs where sessionId = 338 and itemInSession = 4"""
try:
    rows = session.execute(val_query1)
except Exception as e:
    raise Exception(e)

print("Query output - ")
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Query output - 
Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2:
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


Query for the data would look like:
```
select 
    temInSession, 
    artist, 
    song, 
    firstName, 
    lastName 
from 
    user_songs 
where 
    userId = 10 
    and sessionId = 182
```

Plan:
1. As can be seen we require sessionId and userId in our where clause so we know that both those together are gonna make our partition key. Main reason for this would be the fact that multiple user can have the same sessionId, combined together it should give us a unique partition key.
2. Also now that we our results sorted by itemInSession this would have to be our clustering key. 
3. So altogether our primary key should be ((userId, sessionId), itemInSession)
4. The only thing now left is to determine the other columns we require to fulfill the query above which would be temInSession, artist, song, firstName, lastName 

In [18]:
create_query2 = """CREATE TABLE IF NOT EXISTS user_songs
(userId int, sessionId int, artist text, song text, firstName text, lastName text, itemInSession int,
PRIMARY KEY((userId, sessionId), itemInSession))"""

try:
    session.execute(create_query2)
except Exception as e:
    raise Exception(e)                    

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        # wrapping casts to bypass python data defs and feed data to cassandra
        try:
            session.execute(query, (int(userId), int(sessionId), artist, song, firstName, lastName, int(itemInSession)))
        except Exception as e:
            raise Exception(e)    

In [20]:
val_query2 = """select itemInSession, artist, song, firstName, lastName from user_songs where userId = 10 and sessionId = 182"""

try:
    rows = session.execute(val_query2)
except Exception as e:
    raise Exception(e)

print("Query output - ")
for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

Query output - 
0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3:
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


Query for the data would look like:
```
select 
    firstName, 
    lastName 
from 
    app_history 
where 
    song = 'All Hands Against His Own'
```

Plan:
1. This is a unique case where we have only song as where clause, so I think it'd be best to make a primary key out of song and userId to make sure we have it unique across the data. Because user name wouldn't warrant a unique behavior.
2. Besides that we need the user name - (first name, last name).

In [21]:
create_query3 = """CREATE TABLE IF NOT EXISTS app_history
(song text, firstName text, lastName text, userId int,
PRIMARY KEY(song, userId))"""

try:
    session.execute(create_query3)
except Exception as e:
    raise Exception(e)   

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        # wrapping casts to bypass python data defs and feed data to cassandra
        try:
            session.execute(query, (song, firstName, lastName, int(userId)))
        except Exception as e:
            raise Exception(e)    

In [23]:
val_query3 = """select firstName, lastName from app_history where song = 'All Hands Against His Own'"""

try:
    rows = session.execute(val_query3)
except Exception as e:
    raise Exception(e)

print("Query output - ")
for i, row in enumerate(rows):
    print(i+1, row.firstname, row.lastname)

Query output - 
1 Jacqueline Lynch
2 Tegan Levine
3 Sara Johnson


### Drop the tables before closing out the sessions

In [24]:
drop_query1 = "drop table session_songs"
drop_query2 = "drop table user_songs"
drop_query3 = "drop table app_history"
try:
    val1 = session.execute(drop_query1)
    val2 = session.execute(drop_query2)
    val3 = session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()